#### mobileNet

#### importing the libraries

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

#### 1. CONFIGURATION & HYPERPARAMETERS ---

In [6]:

# --- 1. CONFIGURATION & HYPERPARAMETERS ---
# NOTE: Assume DATASET_SPLIT/train and val exist for flow_from_directory
train_dir = "DATASET_SPLIT/train"
val_dir = "DATASET_SPLIT/val"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
MAX_EPOCHS_PHASE_1 = 5
MAX_EPOCHS_PHASE_2 = 7 # Allow more epochs for fine-tuning
LEARNING_RATE_PHASE_1 = 1e-4
LEARNING_RATE_PHASE_2 = 1e-6 # CRITICAL: Much lower for Fine-Tuning
DROPOUT_RATE = 0.4
DENSE_UNITS = 512
CHECKPOINT_PATH = "MobileNetV2.h5"


#### 2. DATA GENERATORS 

In [7]:

train_datagen = ImageDataGenerator(
    rescale=1./255, # Rescale to 0-1 range
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# --- NOTE: Using a placeholder for num_classes since we can't run flow_from_directory ---
# In your actual environment, this will be correctly inferred.
try:
    train_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    val_gen = val_datagen.flow_from_directory(
        val_dir,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    NUM_CLASSES = train_gen.num_classes
except Exception:
    # Mock data for runnable code outside of a real directory structure
    print("Using Mock Data Generator placeholders...")
    NUM_CLASSES = 15 # Placeholder
    
    # --- Mock Data to make the script runnable ---
    X_train_mock = np.random.rand(400, 224, 224, 3).astype('float32')
    y_train_mock = tf.keras.utils.to_categorical(np.random.randint(0, NUM_CLASSES, 400), num_classes=NUM_CLASSES)
    X_val_mock = np.random.rand(100, 224, 224, 3).astype('float32')
    y_val_mock = tf.keras.utils.to_categorical(np.random.randint(0, NUM_CLASSES, 100), num_classes=NUM_CLASSES)

    train_gen = (X_train_mock, y_train_mock)
    val_gen = (X_val_mock, y_val_mock)

Found 46649 images belonging to 16 classes.
Found 11683 images belonging to 16 classes.


#### 3. BUILD THE TRANSFER LEARNING MODEL FUNCTION 

In [8]:
def build_model(num_classes, dropout_rate, dense_units):
    base_model = MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze base for initial training

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax', name="predictions_head")
    ])
    return model, base_model

model, base_model = build_model(NUM_CLASSES, DROPOUT_RATE, DENSE_UNITS)

##### 4.CALLBACKS ---
#### Adjusted patience for stability in Phase 1

In [9]:
early_stop_p1 = EarlyStopping(
    monitor='val_loss',
    patience=8, # Increased patience for stability
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

checkpoint = ModelCheckpoint(
    CHECKPOINT_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

callbacks_p1 = [early_stop_p1, reduce_lr, checkpoint]

#### 5. PHASE 1: TRAIN THE CLASSIFICATION HEAD (Feature Extraction)

In [ ]:
print("\n--- PHASE 1: Training the New Classification Head ---")

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_PHASE_1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Use fit() for mock data, fit_generator() or fit() for actual generators
if isinstance(train_gen, tuple):
    history = model.fit(
        train_gen[0], train_gen[1],
        validation_data=val_gen,
        epochs=MAX_EPOCHS_PHASE_1,
        batch_size=BATCH_SIZE,
        callbacks=callbacks_p1,
        verbose=2
    )
else:
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=MAX_EPOCHS_PHASE_1,
        callbacks=callbacks_p1,
        verbose=2
    )
print("Phase 1 complete. Best weights saved to checkpoint.")


--- PHASE 1: Training the New Classification Head ---
Epoch 1/5


c:\Users\user\anaconda3\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


##### 6. PHASE 2: FINE-TUNING FOR PEAK ACCURACY 

In [ ]:

print("\n--- PHASE 2: Fine-Tuning the Base Model ---")

# Load the best weights saved from Phase 1
if os.path.exists(CHECKPOINT_PATH):
    model.load_weights(CHECKPOINT_PATH)
    print("Loaded best weights from Phase 1.")
else:
    print("Checkpoint not found (likely mock training), proceeding without loading.")

# 6.1 Strategic Unfreezing (Unfreeze the top ~20% of the layers)
base_model.trainable = True

# We keep the first 80% of layers frozen to protect the low-level feature detector
# Total layers in EfficientNetB0 is typically 238
num_unfreeze_layers = int(len(base_model.layers) * 0.20)
for layer in base_model.layers[:-num_unfreeze_layers]:
    layer.trainable = False

print(f"Unfrozen the last {num_unfreeze_layers} layers of the base model for specialization.")
# You can see the total trainable parameters with model.summary()

# 6.2 Re-compile with extremely low learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_PHASE_2), # Use 1e-6
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# New Callbacks list for Phase 2: higher patience for slow fine-tuning
callbacks_p2 = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ModelCheckpoint(CHECKPOINT_PATH, monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-8, verbose=1) # Adjusted LR factor/patience
]

print("Starting Phase 2 Fine-Tuning...")

if isinstance(train_gen, tuple):
    history_fine_tune = model.fit(
        train_gen[0], train_gen[1],
        validation_data=val_gen,
        epochs=MAX_EPOCHS_PHASE_2,
        batch_size=BATCH_SIZE,
        callbacks=callbacks_p2,
        verbose=2
    )
else:
    history_fine_tune = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=MAX_EPOCHS_PHASE_2,
        callbacks=callbacks_p2,
        verbose=2
    )

print("\nTraining complete. The highest validation accuracy is saved in:", CHECKPOINT_PATH)